In [1]:
from snscrape.modules.twitter import *
from tqdm import tqdm

import networkx
import numpy as np
import pandas as pd

In [2]:
query = "url:https://www.rt.com/russia/551440-ukraine-us-financed-biolaboratories/ "

In [3]:
scrapper = TwitterSearchScraper(query).get_items()

In [4]:
df = pd.DataFrame()

for tweet in scrapper:
    data = {
                "url": [tweet.url],
                "id": [tweet.id],
                "username": [tweet.user.username],
                "date_created": [tweet.user.created],
                "tweet_date": [str(tweet.date)],
                "tweet_content": [tweet.rawContent],
                "num_reply": [tweet.replyCount],
                "num_comments": [tweet.quoteCount],
                "num_likes": [tweet.likeCount], 
                "num_retweets": [tweet.retweetCount],
                "num_views": [tweet.viewCount],
                "language": [tweet.lang],
            }
    
    df = pd.concat((df, pd.DataFrame(data)))


In [5]:
df.reset_index(drop=True, inplace=True)

In [7]:
df["tweet_date"] = pd.to_datetime(df["tweet_date"])

In [8]:

dates = np.array([df["tweet_date"].map(pd.Timestamp.timestamp).to_numpy().astype(int).tolist()])

In [9]:
diff = pd.DataFrame(np.absolute(dates - dates.T))

In [10]:
diff = (diff <= 60).astype(int)

In [11]:
diff.columns = df["username"]
diff.index = df["username"]
xs, ys = np.where(diff == 1)

In [12]:
set_of_users = []

for x, y in zip(xs, ys):
    a, b = df["username"][x], df["username"][y]

    if a!=b:
        if a not in set_of_users:
            set_of_users.append(a)
        
        if b not in set_of_users:
            set_of_users.append(b)

In [13]:
user_data = df.groupby("username")["date_created"].unique().to_frame()
user_data.sort_values("date_created", ascending=False)
user_data.reset_index(inplace=True)
user_data = user_data[user_data["username"].map(lambda x: x in set_of_users)]
user_data["date_created"] = pd.to_datetime(user_data["date_created"].map(lambda x: x[0]))
user_data.reset_index(drop=True, inplace=True)
user_data

,username,date_created
0,BrandWe78986136,2021-08-09 06:48:37+00:00
1,CMcdonnelly,2020-09-19 21:26:53+00:00
2,Chris89538871,2017-06-27 11:45:42+00:00
3,DanHill14800006,2021-10-14 20:26:23+00:00
4,DanVin67,2015-11-14 19:45:05+00:00
5,SandraC66126379,2021-08-30 18:03:17+00:00
6,SeekingJustPath,2014-10-18 20:33:37+00:00
7,WesleyKeeley3,2021-10-23 22:11:37+00:00
8,ronmun77,2013-07-11 17:29:14+00:00
9,thamidisetty,2012-04-25 09:05:55+00:00


In [14]:
today = pd.Timestamp.now()
last_n_years = 1
date_since = today - pd.Timedelta(f"{last_n_years*365}D")
date_since = f"{date_since.year}-{date_since.month}-{date_since.day}" # since:2016-09-12
date_since

'2022-2-2'

In [97]:
thresh = 5
user_tweets_dict = {}
last_n_tweets = 10
links = []


for user in tqdm(set_of_users):
    user_tweets = TwitterSearchScraper(f"(from:{user}) filter:links -filter:replies").get_items() # (from:nvidia) filter:links -filter:replies

    user_df = pd.DataFrame()
    temp_links = []

    for i, tweet in enumerate(user_tweets):
        data = {
                    "url": [tweet.url],
                    "id": [tweet.id],
                    "username": [tweet.user.username],
                    "tweet_date": [str(tweet.date)],
                    "tweet_content": [tweet.rawContent],
                    "language": [tweet.lang],
                }
        
        
        links.append(tweet.links[0].url if tweet.links != None else None)
        temp_links.append(tweet.links[0].url if tweet.links != None else None)

        
        
        user_df = pd.concat((user_df, pd.DataFrame(data)))
        # print(i, end="\r")
        if (i+1) % last_n_tweets == 0:
            break

    user_df["link"] = temp_links
    user_df["tweet_date"] = pd.to_datetime(user_df["tweet_date"]).map(pd.Timestamp.timestamp).astype(int)
    user_tweets_dict[user] = user_df

100%|██████████| 12/12 [00:12<00:00,  1.03s/it]


In [98]:
links = pd.Series(links)
links = links.dropna().to_list()
len(links)

56

In [99]:
["from:".join([f"{item} OR " for item in set_of_users])]

['thamidisetty OR from:ronmun77 OR from:SandraC66126379 OR from:Chris89538871 OR from:CMcdonnelly OR from:DanHill14800006 OR from:SeekingJustPath OR from:verdadsiempre12 OR from:DanVin67 OR from:BrandWe78986136 OR from:trevg81003345 OR from:WesleyKeeley3 OR ']

In [91]:
print(f'"{link}" ({"from:".join([f"{item} OR " for item in set_of_users])})')

"https://www.opindia.com/2023/01/mandir-demolished-hindu-women-harassed-children-urdu-and-namaz-media-ignored-plight-of-hindus-in-haldwani/" (thamidisetty OR from:ronmun77 OR from:SandraC66126379 OR from:Chris89538871 OR from:CMcdonnelly OR from:DanHill14800006 OR from:SeekingJustPath OR from:verdadsiempre12 OR from:DanVin67 OR from:BrandWe78986136 OR from:trevg81003345 OR from:WesleyKeeley3 OR )


In [107]:
pd.to_datetime(user_time).map(lambda x: pd.Timestamp.timestamp)

TypeError: <class 'method_descriptor'> is not convertible to datetime

In [134]:
relations = []

final_df = pd.DataFrame(np.zeros((len(set_of_users), len(set_of_users)), dtype=int), columns=set_of_users, index=set_of_users)

for link in tqdm(links):
    related_users = []
    scrapper = TwitterSearchScraper(f'"{link}" ({"from:".join([f"{item} OR " for item in set_of_users])})').get_items()

    user_time = pd.Series([0]*len(set_of_users),index=set_of_users)
    
    for tweet in scrapper:
        user_time[tweet.user.username] = str(tweet.date)
    
    user_time = np.array([pd.to_datetime(user_time).map(pd.Timestamp.timestamp).to_numpy().astype(int).tolist()])
    delta = pd.DataFrame(np.absolute(user_time - user_time.T))
    delta = (delta <= 60).astype(int)
    delta = (delta != 0).astype(int)
    
    delta.columns = set_of_users
    delta.index = set_of_users
    xs, ys = np.where(delta == 1)   

    for x, y in zip(xs, ys):
        a, b = df["username"][x], df["username"][y]

        final_df.loc[a, b] += 1

  0%|          | 0/56 [00:02<?, ?it/s]


KeyError: 'BethguthrieBeth'

In [138]:
# 

[(0, 0),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (1, 1),
 (2, 0),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (3, 0),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 11),
 (4, 0),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (4, 11),
 (5, 0),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (5, 10),
 (5, 11),
 (6, 0),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 6),
 (6, 7),
 (6, 8),
 (6, 9),
 (6, 10),
 (6, 11),
 (7, 0),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (8, 0),
 (8, 2),
 (8, 3),
 (8, 4),
 (8, 5),
 (8, 6),
 (8, 7),
 (8, 8),
 (8, 9),
 (8, 10),
 (8, 11),
 (9, 0),
 (9, 2),
 (9, 3),
 (9, 4),
 (9, 5),
 (9, 6),
 (9, 7),
 (9, 8),
 (9, 9),
 (9, 10),
 (9, 11),
 (10, 0),
 (10, 2),
 (10, 3),
 (10, 4),
 (10, 5),
 (10, 6),
 (10, 7),
 (10, 8),
 (

In [133]:
np.where(delta == 1)

(array([], dtype=int64), array([], dtype=int64))

In [130]:
delta = pd.DataFrame(np.absolute(user_time - user_time.T))
delta = ((delta <= 60) & (delta != 0)).astype(int)

In [118]:
delta

,0
thamidisetty,0
ronmun77,0
SandraC66126379,0
Chris89538871,0
CMcdonnelly,0
DanHill14800006,0
SeekingJustPath,0
verdadsiempre12,0
DanVin67,0
BrandWe78986136,0
